<a id='Q0'></a>
<center> <h1> Notebook 2: Advanced Visualization </h1> </center>
<p style="margin-bottom:1cm;"></p>
<center><strong>Angela Niederberger, 2022</strong></center>
<p style="margin-bottom:1cm;"></p>

<div style="background:#EEEDF5;border-top:0.1cm solid #EF475B;border-bottom:0.1cm solid #EF475B;">
    <div style="margin-left: 0.5cm;margin-top: 0.5cm;margin-bottom: 0.5cm;color:#303030">
        <p><strong>Goal:</strong> Short description of the aim of the notebook</p>
        <strong> Outline:</strong>
        <a id='P0' name="P0"></a>
        <ol>
            <li> <a style="color:#303030" href='#I'>Introduction </a> </li>
            <li> <a style="color:#303030" href='#SU'>Set up</a></li>
            <li> <a style="color:#303030" href='#P1'>Exploratory Data Analysis</a></li>
            <li> <a style="color:#303030" href='#P2'>Visualization</a></li>
            <li> <a style="color:#303030" href='#P3'>Part 3</a></li>
            <li> <a style="color:#303030" href='#CL'>Conclusion</a></li>
            <li> <a style="color:#303030" href='#AP'>Appendix</a></li>
        </ol>
        <strong>Keywords:</strong> data cleaning, NLP, ... list of keywords.
    </div>
</div>
</nav>

<a id='I' name="I"></a>
## [Introduction](#P0)

Here you could write an introduction to the topics. Everything that is written here will appear in the live coding template notebook

<a id='SU' name="SU"></a>
## [Set up](#P0)

### Packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import os
import json

### Magic Commands

In [2]:
# %load_ext lab_black


%matplotlib inline
%config Completer.use_jedi = False

### Global Parameters Setting

In [ ]:
plt.style.use("seaborn")
sns.set_theme(style="ticks")

### User-Dependent Variables

## ***Need to make token hidden*** ##

In [3]:
raw_data = "../data/raw"
proc_data = "../data/processed"
mapbox_access_token = "pk.eyJ1IjoiampvbmVzMSIsImEiOiJjbDdzaG9xcTkwODM0M3BtbXp2OW5oa29mIn0.d1NUWF43CRbDcCZ4B-V7UA"
quali_colorscale = px.colors.qualitative.Bold

os.getcwd()

'C:\\Users\\JJones\\Desktop\\00_PythonWIP\\swiss_rents-master\\notebooks'

### Helper Functions

Some helpful functions either from the previous notebook or from elsewhere.

In [4]:
def clean_swiss_rents_data(df):
    # Turn PLZ_Ort into two independent columns
    df["Ort"] = df["PLZ_Ort"].str.split(" ", 1).str[1]
    df["PLZ"] = df["PLZ_Ort"].str.split(" ", 1).str[0].astype("int")

    # Remove all the unnecessary commas in KT
    ###df["KT,"] = df["KT,"].str.replace(",", "")
    ###df.rename(columns={"KT,": "KT"}, inplace=True)

    # Turn KT into actual Canton name
    cantons_dict = {
        "TG": "Thurgau",
        "GR": "Graubünden",
        "LU": "Luzern",
        "BE": "Bern",
        "VS": "Valais",
        "BL": "Basel-Landschaft",
        "SO": "Solothurn",
        "VD": "Vaud",
        "SH": "Schaffhausen",
        "ZH": "Zürich",
        "AG": "Aargau",
        "UR": "Uri",
        "NE": "Neuchâtel",
        "TI": "Ticino",
        "SG": "St. Gallen",
        "GE": "Genève",
        "GL": "Glarus",
        "JU": "Jura",
        "ZG": "Zug",
        "OW": "Obwalden",
        "FR": "Fribourg",
        "SZ": "Schwyz",
        "AR": "Appenzell Ausserrhoden",
        "AI": "Appenzell Innerrhoden",
        "NW": "Nidwalden",
        "BS": "Basel-Stadt",
    }
    df["Kanton"] = df["KT"].map(cantons_dict)

    return df

In [5]:
def create_new_features(df):
    # Add rent categories
    df["sqm_preis"] = df["Mietpreis_Brutto"] / df["Fläche"]
    descr_rents = df["sqm_preis"].describe()

    quantiles = df["sqm_preis"].quantile(q=[0.15, 0.5, 0.85])
    cheap = quantiles[0.15]
    average = quantiles[0.5]
    expensive = quantiles[0.85]

    df["Miete_Kategorie"] = np.where(
        df["sqm_preis"] < cheap,
        0,
        np.where(
            (df["sqm_preis"] >= cheap) & (df["sqm_preis"] < average),
            1,
            np.where(
                (df["sqm_preis"] >= average) & (df["sqm_preis"] < expensive),
                2,
                3,
            ),
        ),
    )

    # Hover strings
    df["hover_strings_scatter"] = [
        f"Address: {street}, {place},<br>Rooms: {rooms}, Floor Space: {round(size)}m²,<br>Rent: CHF {rent}"
        for street, place, rooms, size, rent in zip(
            df["Adresse"],
            df["Ort"],
            df["Zimmer"],
            df["Fläche"],
            df["Mietpreis_Brutto"],
        )
    ]

    return df

<a id='P2' name="P2"></a>
## [Data Pre-processing](#P0)

This section shows the preparations required to get a clean dataset, which can then be used for advanced visualizations with Plotly.

### Loading the Data

In [ ]:
rents_df = pd.read_csv(f"{raw_data}/mietinserate_v1.csv", sep=";")
rents_df = clean_swiss_rents_data(rents_df)
rents_df.head()

### Converting Coordinates

The following script was used to request GPS coordinates from the swisstopo API. They are based on the coordinates ``GKODE`` and ``GKODN`` which are coordinates on a Swiss system (LV95). The loop takes 2-3 hours to complete, so I've commented this code out.

In [ ]:
"""
long = []
lat = []

for i in range(len(rents_df)):    
    r = requests.get(f'http://geodesy.geo.admin.ch/reframe/lv95towgs84?easting={rents_df.loc[i, "GKODE"]}&northing={rents_df.loc[i, "GKODN"]}&format=json')
    coord_dict = json.loads(r.content)
    long.append(coord_dict["easting"])
    lat.append(coord_dict["northing"])
    
rents_df["long"] = long
rents_df["lat"] = lat
"""

The coordinates were saved in the file that I'm loading below. I'll just transfer them into the dataframe created before, for the sake of simplicity.

In [ ]:
df_proc_old = pd.read_csv(f"{proc_data}/rents_with_coords_clean.csv")

## **Had to correct first line of code below from: rents_df["long"] = df_proc_old["long"] to what is currently there** ##

In [ ]:
rents_df["lon"] = df_proc_old["lon"]
rents_df["lat"] = df_proc_old["lat"]
rents_df

### Handling Missing Values

Missing values can create problems when plotting, so they are either filled or removed below. I'm also dropping columns that will not be useful.

In [ ]:
cols_to_drop = [
    "ID",
    "Jahr",
    "Quartal",
    "Wohnungstyp",
    "Quadratmeterpreis_Brutto",
    "PLZ_Ort",
    "Bezugsdatum",
    "GKODE",
    "GKODN",
    "KT",
]

rents_df.drop(columns=cols_to_drop, inplace=True)
rents_df.dropna(inplace=True)
print(rents_df.shape)
rents_df.head()

### Rows with Errors

Some rows have errors or inconsistencies, like for example if the place is 'Zürich' but the canton is 'Bern'. We need to remove these.

In [ ]:
rents_df.drop(
    rents_df[
        (rents_df["Ort"] == "Zürich") & (rents_df["Kanton"] != "Zürich")
        | (rents_df["Ort"] == "Basel") & (rents_df["Kanton"] != "Basel-Stadt")
    ].index,
    inplace=True,
)
rents_df.shape

### Adding Features for Plotting

In [ ]:
rents_df = create_new_features(rents_df)
rents_df.head()

In [ ]:
rents_df["Miete_Kategorie"].value_counts()

In [ ]:
rents_df.sort_values("Miete_Kategorie", inplace=True)
rents_df.to_csv(f"{proc_data}/rents_with_coords_clean.csv", index=False)

<a id='P2' name="P2"></a>
## [Advanced Visualization](#P0)

The goal of this section is to create a couple of different visualizations of this data set that can be used for a Streamlit app. The app should also be able to take some user inputs and visualize the data accordingly

In [ ]:
df_proc = pd.read_csv(f"{proc_data}/rents_with_coords_clean.csv")
df_proc.head()

<div style="background:#EEEDF5;border:0.1cm solid #00BAE5;color:#303030">
    <div style="margin: 0.2cm 0.2cm 0.2cm 0.2cm">
        <b style="color:#00BAE5">Note:</b>
        This is the processed data file saved from the previous section. It contains GPS coordinates requested from the swisstopo API (columns 'long' and 'lat'). Also, some cleaning operations have already been performed on this data. It is only meant for plotting (not modelling).
    </div>
</div>

In [ ]:
# Sampling to reduce the size of the data file

df_proc_small = df_proc.sample(frac=0.1)
df_proc_small.info()

It would be nice to plot the canton boundaries, in addition to the apartment locations, so the code below reads in a geojson file containing the required geographical information.

In [ ]:
with open(f"{raw_data}/georef-switzerland-kanton.geojson") as response:
    cantons = json.load(response)

cantons["features"][0]["properties"]

### Scattermap

In [ ]:
def set_up_scattermap():
    colors = px.colors.qualitative.Bold

    traces = [
        "cheapest",
        "below average",
        "above average",
        "most expensive",
    ]

    go_figure = go.Figure()

    return colors, traces, go_figure


def add_scattermap_traces(df, go_figure, colors, traces):
    for cat, df_grouped in df.groupby("Miete_Kategorie"):
        go_figure.add_trace(
            go.Scattermapbox(
                long=df_grouped["long"],
                lat=df_grouped["lat"],
                mode="markers",
                marker=go.scattermapbox.Marker(size=5, color=colors[cat], opacity=0.5),
                text=df_grouped["hover_strings_scatter"],
                hovertemplate="%{text}<extra></extra>",
                name=traces[cat],
                legendgroup=str(cat),
            )
        )
    return go_figure


def add_scattermap_layout(go_figure, mapbox_token):
    go_figure.update_layout(
        margin={"r": 0, "t": 45, "l": 0, "b": 0},
        width=1100,
        height=660,
        hovermode="closest",
        mapbox=dict(
            accesstoken=mapbox_token,
            bearing=0,
            center=go.layout.mapbox.Center(lat=46.8, long=8.3),
            pitch=0,
            zoom=7,
            layers=[{"source": cantons, "type": "line", "line_width": 1}],
        ),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            #   y=0.54,
            xanchor="center",
            x=0.5,
            font_size=16,
            itemsizing="constant",
        ),
        template="simple_white",
        title="Location of Listed Apartments",
        title_font_size=25,
    )
    return go_figure


def build_scattermap(df, mapbox_token):

    colors, traces, go_figure = set_up_scattermap()
    go_figure = add_scattermap_traces(df, go_figure, colors, traces)
    go_figure = add_scattermap_layout(go_figure, mapbox_token)

    return go_figure

## **Need mapbox token** ##

In [ ]:
map_fig = build_scattermap(df_proc_small, mapbox_access_token)

map_fig

### Combining Figures: Scatter plot and bar plot

In [ ]:
## Functions for the combined plot


def set_up_subplots():
    colors = px.colors.qualitative.Bold

    go_figure = make_subplots(
        rows=1,
        cols=2,
        subplot_titles=(
            "Apartments by Size and Rent",
            "Apartments per Canton",
        ),
        column_widths=[2, 1],
    )
    return go_figure, colors


def add_scatter_traces(df, go_figure, colors):
    for cat, df_grouped in df.groupby("Miete_Kategorie"):
        go_figure.add_trace(
            go.Scatter(
                x=df_grouped["Fläche"],
                y=df_grouped["Mietpreis_Brutto"],
                mode="markers",
                marker={"color": colors[cat], "opacity": 0.5},
                text=df_grouped["hover_strings_scatter"],
                hovertemplate="%{text}<extra></extra>",
                #   name=traces[cat],
                legendgroup=str(cat),
                showlegend=False,
            ),
            row=1,
            col=1,
        )
    return go_figure


def add_barplot_traces(df, go_figure, colors):
    df_grouped = (
        df.groupby(["Kanton", "Miete_Kategorie"])
        .size()
        .unstack(level=-1)
        .fillna(0)
        .sort_values("Kanton", ascending=False)
        .reset_index()
    )
    df_grouped["Total_Kanton"] = df_grouped.iloc[:, 1:].sum(axis=1)

    for cat in df["Miete_Kategorie"].unique():
        hover_strings = [
            f"{round(num_per_canton / total_canton * 100, 2)}%"
            for num_per_canton, total_canton in zip(
                df_grouped.loc[:, cat], df_grouped["Total_Kanton"]
            )
        ]
        go_figure.add_trace(
            go.Bar(
                y=df_grouped["Kanton"],
                x=df_grouped.loc[:, cat],
                orientation="h",
                marker_color=colors[cat],
                text=hover_strings,
                hovertemplate="%{text}<extra></extra>",
                #   name=traces[cat],
                legendgroup=str(cat),
                showlegend=False,
            ),
            row=1,
            col=2,
        )
    return go_figure


def define_figure_layout(go_figure):
    go_figure.update_layout(
        margin={"r": 0, "t": 45, "l": 0, "b": 0},
        width=875,
        height=550,
        hovermode="closest",
        template="simple_white",
        barmode="stack",
    )
    return go_figure


def build_combined_figure(df):
    go_figure, colors = set_up_subplots()

    # Scatter plot
    go_figure = add_scatter_traces(df, go_figure, colors)

    # Bar plot
    go_figure = add_barplot_traces(df, go_figure, colors)

    # Subplot title font size
    go_figure.layout.annotations[0].update(font_size=25, x=0.17, y=1.01)
    go_figure.layout.annotations[1].update(font_size=25, x=0.8, y=1.01)

    # Axis Labels
    go_figure.update_xaxes(
        title={"text": "Floor Space (m²)", "font_size": 16}, row=1, col=1
    )
    go_figure.update_yaxes(title={"text": "Rent (CHF)", "font_size": 16}, row=1, col=1)

    go_figure.update_xaxes(
        title={"text": "Number of Listings", "font_size": 16}, row=1, col=2
    )

    # Layout
    go_figure = define_figure_layout(go_figure)

    return go_figure

In [ ]:
joint_fig = build_combined_figure(df=df_proc_small)

joint_fig.show()

<div style="border-top:0.1cm solid #EF475B"></div>
    <strong><a href='#Q0'><div style="text-align: right"> <h3>End of this Notebook.</h3></div></a></strong>